# Homework 3: Language Modelling in Hangman

Student Name:  Swapnil Shailee

Student ID:  952247

## General info

<b>Due date</b>:  Friday, 17 May 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models. Your objective is to create an automatic player which makes the fewest mistakes.

## The Hangman Game (*No implementation is needed*)

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. 

In [30]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided gusser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [31]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('Enter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to True. When submitting your solution, set to False so we can automatically run the whole notebook using `Run All`.

In [63]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [33]:
if interactive:
  
    hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
w
Guess is w
Good guess: w _ _ _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
h
Guess is h
Good guess: w h _ _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
a
Guess is a
Good guess: w h a _ _ _ _ _
You have 8 attempts remaining.
Enter your guess:
t
Guess is t
Good guess: w h a t _ _ _ _
You have 8 attempts remaining.
Enter your guess:
e
Guess is e
Good guess: w h a t e _ e _
You have 8 attempts remaining.
Enter your guess:
v
Guess is v
Good guess: w h a t e v e _
You have 8 attempts remaining.
Enter your guess:
r
Guess is r
Good guess: w h a t e v e r
Congratulations, you won.


## 1. Preparing Test Set and Training Set (1 mark)

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the algorithm. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown` and the `words` method, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(1 mark)

In [38]:
from nltk.corpus import brown
import numpy as np
import nltk

# word_set stores all the unique word types of the Brown corpus
word_set = []
# test_set stores 1000 word types for testing purpose
test_set = []
# training_set stores the rest word types for training purpose
training_set = []

###
# Your answer BEGINS HERE
###
# storing word types in dataset that are alphabetic and lowercased
dataset = nltk.corpus.brown.words()
for word in dataset:
    if word.isalpha():
        term =word.lower()
        if term not in word_set:
            word_set.append(term)
np.random.shuffle(word_set)
#from start to 1000 word types of the word set
test_set=word_set[:1000]
#from 1000 to rest of word types of the word set
training_set=word_set[1000:]




###
# Your answer ENDS HERE
###

print(len(word_set))
print(len(test_set))
print(len(training_set))

40234
1000
39234


<b>For your testing:</b>

In [39]:
assert(len(word_set) > 35000 and len(word_set) < 45000)

In [40]:
assert(len(test_set) == 1000)

In [41]:
assert(len(training_set) + len(test_set) == len(word_set))

In [42]:
if interactive:
  
  
    hangman(np.random.choice(test_set), human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ _ _ _ length 11
You have 8 attempts remaining.
Enter your guess:
d
Guess is d
Sorry, try again.
You have 7 attempts remaining.
Enter your guess:
a
Guess is a
Sorry, try again.
You have 6 attempts remaining.
Enter your guess:
i
Guess is i
Good guess: i _ _ _ _ _ _ _ i _ _
You have 6 attempts remaining.
Enter your guess:
n
Guess is n
Good guess: i n _ _ n _ _ _ i _ _
You have 6 attempts remaining.
Enter your guess:
o
Guess is o
Good guess: i n _ o n _ _ _ i _ _
You have 6 attempts remaining.
Enter your guess:
m
Guess is m
Sorry, try again.
You have 5 attempts remaining.
Enter your guess:
r
Guess is r
Good guess: i n _ o n _ r _ i _ _
You have 5 attempts remaining.
Enter your guess:
j
Guess is j
Sorry, try again.
You have 4 attempts remaining.
Enter your guess:
k
Guess is k
Sorry, try again.
You have 3 attempts remaining.
Enter your guess:
r
Guess is r
Already guessed this before.
You have 2 attempts remaining.
Enter your guess:
e
Guess is 

## 2. Simple Guesser: Random Guessing (1 mark)

<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

(1 mark)

In [43]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [45]:
import string
import random

def random_guesser(mask, guessed, **kwargs):
    """
        This function implements a random guesser. It returns the random guess. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    #alphabet_set stores all the lowercase alphabets and letter is chosen randomly from the set
    alphabet_set = string.ascii_lowercase
    letter=random.choice(alphabet_set)
    #if letter is not in guessed then letter is returned otherwise random guesser is called
    if letter not in guessed:
        return letter
    else:
        return random_guesser(mask, guessed, **kwargs)

    ###
    # Your answer ENDS HERE
    ###

# uncomment to run a single hangman game with output shown (useful for debugging)
hangman(np.random.choice(test_set), random_guesser, 10, True)

result = test_guesser(random_guesser)
print()
print("Average number of incorrect guesses: ", result)

Starting hangman game. Target is _ _ _ length 3
You have 10 attempts remaining.
Guess is d
Sorry, try again.
You have 9 attempts remaining.
Guess is i
Sorry, try again.
You have 8 attempts remaining.
Guess is m
Sorry, try again.
You have 7 attempts remaining.
Guess is n
Sorry, try again.
You have 6 attempts remaining.
Guess is p
Sorry, try again.
You have 5 attempts remaining.
Guess is z
Sorry, try again.
You have 4 attempts remaining.
Guess is j
Sorry, try again.
You have 3 attempts remaining.
Guess is a
Sorry, try again.
You have 2 attempts remaining.
Guess is o
Good guess: _ _ o
You have 2 attempts remaining.
Guess is h
Sorry, try again.
You have 1 attempts remaining.
Guess is c
Sorry, try again.
Out of guesses. The word was leo

Average number of incorrect guesses:  16.645


<b>For your testing:</b>

In [46]:
assert(result > 10 and result < 20)

## 3. Your First AI Guesser: Unigram Guesser (1 mark)

**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

Hint: It should be much lower than random guessing.

(1 mark)

In [47]:
from collections import Counter
import math

# unigram_counts stores the frequencies of characters over all training words


###
# Your answer BEGINS HERE
###
#loop through all characters of the training set
unigram_counts = Counter()
for word in training_set:
    for letter in word:
        unigram_counts[letter]+=1

###
# Your answer ENDS HERE
###

prob_count = Counter()

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """
        This function implements a unigram guesser. It returns a guess based on the unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    #sums the unigram counts and calculates the probability for all letters
    sum_count = float(sum(unigram_counts.values()))
    for i in range(len(unigram_counts)):
        prob_count[unigram_counts.most_common()[i][0]] = unigram_counts.most_common()[i][1]/sum_count
    #selects the most common letter based on probability and returns if not guessed
    i=0
    letter=prob_count.most_common()[i][0]
    while(letter in guessed):
        i+=1
        letter=prob_count.most_common()[i][0]
    
    return letter
    
    ###
    # Your answer ENDS HERE
    ###

hangman(np.random.choice(test_set), unigram_guesser, 10, True)

result = test_guesser(unigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Starting hangman game. Target is _ _ _ _ _ length 5
You have 10 attempts remaining.
Guess is e
Sorry, try again.
You have 9 attempts remaining.
Guess is i
Sorry, try again.
You have 8 attempts remaining.
Guess is a
Sorry, try again.
You have 7 attempts remaining.
Guess is s
Good guess: _ _ s _ s
You have 7 attempts remaining.
Guess is n
Sorry, try again.
You have 6 attempts remaining.
Guess is r
Sorry, try again.
You have 5 attempts remaining.
Guess is t
Good guess: _ _ s t s
You have 5 attempts remaining.
Guess is o
Good guess: _ o s t s
You have 5 attempts remaining.
Guess is l
Sorry, try again.
You have 4 attempts remaining.
Guess is c
Sorry, try again.
You have 3 attempts remaining.
Guess is d
Sorry, try again.
You have 2 attempts remaining.
Guess is u
Sorry, try again.
You have 1 attempts remaining.
Guess is m
Sorry, try again.
Out of guesses. The word was hosts

Average number of incorrect guesses:  10.337


<b>For your testing:</b>

In [48]:
assert(result > 5 and result < 15)

## 4. Your Second AI Guesser: Length-based Unigram Guesser (1 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having a *different* unigram model for each length of the words. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. You need to decide on how to handle this situation.

(1 mark)

In [51]:
from collections import defaultdict
from string import ascii_lowercase
# unigram_counts_by_length stores a dictionary, mapping word length to the frequencies of characters of words with that word length
unigram_counts_by_length = defaultdict(Counter)

###
# Your answer BEGINS HERE
###
#function to calculate unigram probability
def get_unigram_prob(length = -1):
    unigram_probability = {}
    letter_freq = {}
    for word in training_set:
        if length == -1 or len(word)==length:
            for char in word:
                letter_freq[char] = float(letter_freq.get(char,0)+1)
    
    totalfreq = float(sum(letter_freq.values()))
    
    for key, value in sorted(letter_freq.items(), key=lambda kv: (kv[1], kv[0])):
        unigram_probability[key] = float(value/totalfreq)
    return unigram_probability

#function to calculate unigram frequency
def getUnigramFreq(Length):
    freq = {}
    for word in training_set:
        if len(word)== Length:
            for char in word:
                freq[char] = float(freq.get(char,0)+1)
    return freq

unigram_models = []
unigram_dict ={}
max_wordlength = 0;
for word in training_set:
    if len(word) > max_wordlength:
        max_wordlength = len(word)
        
for wordlength in range(max_wordlength+1):
    if wordlength!=0:
        unigram_models.append(get_unigram_prob(length = wordlength))
        unigram_dict[wordlength]=getUnigramFreq(wordlength)

unigram_models.append(get_unigram_prob())       
unigram_counts_by_length = unigram_dict
sorted_alphabets = []
###
# Your answer ENDS HERE
###


lengths = sorted(unigram_counts_by_length.keys())
print(lengths)
max_length = lengths[-1] + 1
print()
print(unigram_counts_by_length)

if len(word) <= max_wordlen + 1:
    for key, value in sorted(unigram_models[len(word)-1].items(), key=lambda kv: (kv[1], kv[0]),reverse = True):
        sorted_alphabets.append(key)   
else:
    for key, value in sorted(unigram_models[-1].items(), key=lambda kv: (kv[1], kv[0]),reverse = True):
        sorted_alphabets.append(key)  


def unigram_length_guesser(mask, guessed, counts=unigram_counts_by_length):
    """
        This function implements a length-based unigram guesser. It returns a guess based on the length-based unigram model. 
    """
    ###
    # Your answer BEGINS HERE
    ###
    
    i = 0
    #most probable alphabet
    letter = sorted_alphabets[0]
    while letter in guessed:
        letter = sorted_prob_alphabets[i]
        i = i+1
    return letter
    
    
    
    ###
    # Your answer ENDS HERE
    ###

hangman(np.random.choice(test_set), unigram_length_guesser, 10, True)

result = test_guesser(unigram_length_guesser)
print()
print("Average number of incorrect guesses: ", result)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

{1: {'r': 1.0, 'b': 1.0, 'x': 1.0, 'f': 1.0, 'u': 1.0, 'j': 1.0, 'w': 1.0, 'y': 1.0, 'l': 1.0, 'n': 1.0, 'h': 1.0, 'z': 1.0, 'o': 1.0, 'v': 1.0, 'i': 1.0, 'm': 1.0, 'k': 1.0, 'g': 1.0, 'q': 1.0, 'd': 1.0, 'p': 1.0, 's': 1.0, 'a': 1.0, 'c': 1.0, 'e': 1.0, 't': 1.0}, 2: {'s': 18.0, 'w': 7.0, 'e': 21.0, 'n': 10.0, 'v': 7.0, 'h': 11.0, 't': 9.0, 'i': 16.0, 'r': 9.0, 'k': 5.0, 'u': 8.0, 'o': 17.0, 'b': 8.0, 'y': 6.0, 'a': 21.0, 'd': 9.0, 'x': 3.0, 'm': 19.0, 'g': 6.0, 'p': 11.0, 'l': 13.0, 'q': 2.0, 'f': 10.0, 'c': 10.0, 'j': 5.0, 'z': 1.0}, 3: {'n': 115.0, 'u': 101.0, 't': 115.0, 'e': 197.0, 'l': 77.0, 'i': 127.0, 'r': 100.0, 'p': 100.0, 'o': 181.0, 'v': 24.0, 's': 112.0, 'b': 92.0, 'a': 235.0, 'w': 73.0, 'm': 103.0, 'h': 78.0, 'd': 97.0, 'q': 6.0, 'f': 52.0, 'g': 76.0, 'y': 78.0, 'c': 69.0, 'k': 31.0, 'z': 10.0, 'j': 29.0, 'x': 23.0}, 4: {'s': 696.0, 'l': 559.0, 'a': 875.0, 'm': 300.0, 'g': 222.0, 'i': 543.0,

<b>For your testing:</b>

In [52]:
assert(result > 5 and result < 15)

## 5. Your Third AI Guesser: Bigram Guesser (1 mark)

**Instructions:** Now for the next challenge, using a *bigram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

You should develop a *bigram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with a sentinel symbol `$`.) You should use *linear interpolation* to smooth between the higher order and lower order models, and you will have to decide how to weight each component (be reminded that all probabilities should sum to 1).

Your bigram guesser should apply your language model to each blank position in the secret word by using the left context as is known. E.g., in the partial word `$ _ e c _ e _ _` we know the left context for the first three blanks, but have no known left context for the last blank. Using a bigram language model, we are able to apply it to the first three blanks only. You should then select the character with the highest probability of predicting the most number of correct entries over the blanks. 

Do you see any improvement over the unigram guessers above?

(1 mark)

In [57]:
###
# Your answer BEGINS HERE
###
bigram_counts = defaultdict(Counter) 

def convert_word(word):
    return '^' + word

ngram_counts = []
ngram_counts.append(Counter()) # unigram_counts
ngram_counts.append(bigram_counts)  # bigram_counts
for word in training_set:
    word = convert_word(word)
    for i in range(1, len(word)):
        ngram_counts[0][word[i]] += 1  #unigram_counts increment
        if i >= 1:
            ngram_counts[1][word[i-1]][word[i]] += 1  #bigram_counts increment


#interploation function
def get_char_prob_interp(prev_chars, char, n_gram, ngram_counts, lambdas):
    interp_n_gram_counts = [0] * 5
    for i in range(n_gram, 0, -1):
        if i != 1:
            char_i_gram_count = ngram_counts[i-1][prev_chars[-(i-1):]][char]
            prev_chars_count = float( sum( ngram_counts[i-1][prev_chars[-(i-1):]].values()) + 1e-6)
            interp_n_gram_counts[i-1] = char_i_gram_count / prev_chars_count * lambdas[i-1]
        else:
            interp_n_gram_counts[i-1] = ngram_counts[i-1][char] / float( sum( n_gram_counts[i-1].values())) * lambdas[i-1]
    return math.log(sum(interp_n_gram_counts))
    
    

        

###
# Your answer ENDS HERE
###

def bigram_guesser(mask, guessed, counts= ngram_counts, **kwargs): # add extra default arguments if needed
    """
        This function implements a bigram guesser. It returns a guess based on the bigram model using linear interpolation.
    """
    ###
    # Your answer BEGINS HERE
    ###
   
    n_gram = 2
    labmdas = [[1], [0.1,0.9], [0.01,0.09,0.9], [0.001,0.009,0.09,0.9], [0.0001,0.0009,0.009,0.09,0.9]]
    mask = ''.join(mask)
    mask = convert_word(mask)
    
    all_prev_chars = []
    prev_chars = ""
    for char in mask:
        if char != '_':
            prev_chars += char
        else:
            all_prev_chars.append(prev_chars)
            prev_chars = ""
            
    choices = set(string.ascii_lowercase).difference(guessed)
    maxsum_char_prob_interp = float("-inf")
    choice = ''
    for char in choices:
        sum_char_prob_interp = 0
        for prev_chars in all_prev_chars:
            char_n_gram = n_gram if ((len(prev_chars) + 1) > n_gram) else (len(prev_chars) + 1)
            sum_char_prob_interp += get_char_prob_interp(str(prev_chars), char, char_n_gram, n_gram_counts, labmdas[char_n_gram-1])
        if sum_char_prob_interp > maxsum_char_prob_interp:
            maxsum_char_prob_interp = sum_char_prob_interp
            choice = char #best choice
    
    return choice


      
   
    
    ###
    # Your answer ENDS HERE
    ###

hangman(np.random.choice(test_set), bigram_guesser, 10, True)

result = test_guesser(bigram_guesser)
print()
print("Average number of incorrect guesses: ", result)

Starting hangman game. Target is _ _ _ _ _ _ _ length 7
You have 10 attempts remaining.
Guess is e
Sorry, try again.
You have 9 attempts remaining.
Guess is s
Sorry, try again.
You have 8 attempts remaining.
Guess is i
Good guess: _ _ _ _ i _ _
You have 8 attempts remaining.
Guess is n
Good guess: _ _ _ _ i n _
You have 8 attempts remaining.
Guess is a
Good guess: _ a _ _ i n _
You have 8 attempts remaining.
Guess is t
Sorry, try again.
You have 7 attempts remaining.
Guess is c
Sorry, try again.
You have 6 attempts remaining.
Guess is d
Good guess: _ a _ d i n _
You have 6 attempts remaining.
Guess is g
Good guess: _ a _ d i n g
You have 6 attempts remaining.
Guess is r
Sorry, try again.
You have 5 attempts remaining.
Guess is l
Good guess: _ a l d i n g
You have 5 attempts remaining.
Guess is p
Sorry, try again.
You have 4 attempts remaining.
Guess is b
Good guess: b a l d i n g
Congratulations, you won.

Average number of incorrect guesses:  8.657


<b>For your testing:</b>

In [58]:
assert(result < 13)

## 6. Your Own AI Guesser (1 mark)

**Instructions:** You should try to develop a more effective AI, `my_ai_guesser`, for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, and using ngram models. Ensure you report the test performance of your guesser. Have fun!

You will be marked based on the explanation of your approach and its accuracy. 

(1 mark) 

In [62]:
###
# Your answer BEGINS HERE
###
#list storing n gram counts
ngram_counts = []
n_gram_counts.append(Counter())             #unigram_counts
n_gram_counts.append(defaultdict(Counter))  #bigram_counts
n_gram_counts.append(defaultdict(Counter))  #trigram_counts
n_gram_counts.append(defaultdict(Counter))  #quadgram_counts
n_gram_counts.append(defaultdict(Counter))  #quingram_counts

def convert_word(word):
    return '^' + word

for word in training_set:
    word = convert_word(word)
    for i in range(1, len(word)):
            n_gram_counts[0][word[i]] += 1   #1-gram increments
            if i >= 1:
                n_gram_counts[1][word[i-1]][word[i]] += 1  #2-gram increments
            if i >= 2:
                n_gram_counts[2][word[i-2:i]][word[i]] += 1  #3-gram increments
            if i >= 3:
                n_gram_counts[3][word[i-3:i]][word[i]] += 1  #4-gram increments
            if i >= 4:
                n_gram_counts[4][word[i-4:i]][word[i]] += 1  #5-gram increments

#interpolation function
def get_char_prob_interp(prev_chars, char, n_gram, n_gram_counts, lambdas):
    interp_n_gram_counts = [0] * 5
    for i in range(n_gram, 0, -1):
        if i != 1:
            char_i_gram_count = n_gram_counts[i-1][prev_chars[-(i-1):]][char]
            prev_chars_count = float( sum( n_gram_counts[i-1][prev_chars[-(i-1):]].values()) + 1e-6)  # sum of ngram counts + 1 * 10**-6
            interp_n_gram_counts[i-1] = char_i_gram_count / prev_chars_count * lambdas[i-1]
        else:
            interp_n_gram_counts[i-1] = n_gram_counts[i-1][char] / float( sum( n_gram_counts[i-1].values())) * lambdas[i-1]
    return math.log(sum(interp_n_gram_counts))

# 5-gram guesser 
def my_ai_guesser(mask, guessed, **kwargs):
    n_gram = 5 
    #lambda values 
    lambdas = [[1], [0.1,0.9], [0.01,0.09,0.9], [0.001,0.009,0.09,0.9], [0.0001,0.0009,0.009,0.09,0.9]]
    mask = ''.join(mask)
    mask = convert_word(mask)
    
    all_prev_chars = []
    prev_chars = ""
    for char in mask:
        #previous character is filled
        if char != '_':
            prev_chars += char
        else:
            all_prev_chars.append(prev_chars)
            prev_chars = ""
        
    choices = set(string.ascii_lowercase).difference(guessed)
    #setting max to infinity
    maxsum_char_prob_interp = float("-inf")
    choice = ''
    for char in choices:
        sum_char_prob_interp = 0
        for prev_chars in all_prev_chars:
            char_n_gram = n_gram if ((len(prev_chars) + 1) > n_gram) else (len(prev_chars) + 1)
            sum_char_prob_interp += get_char_prob_interp(str(prev_chars), char, char_n_gram, n_gram_counts, lambdas[char_n_gram-1])
        if sum_char_prob_interp > maxsum_char_prob_interp:
            maxsum_char_prob_interp = sum_char_prob_interp
            choice = char #best choice of character
    
    return choice
    
    return random_guesser(mask, guessed)

###
# Your answer ENDS HERE
###

result = test_guesser(my_ai_guesser)
print()
print("Average number of incorrect guesses: ", result)


Average number of incorrect guesses:  7.49


**Instructions:** Explain your approach and discuss your result below. Please keep it brief.

The AI guesser is an extention of the bigram guesser. 5-gram approach have been implemented and interpolation method is used for smoothening the model hence the previous 4 characters have also been taken into consideration to formulate the guesser.Interpolation method combines N-grams of different orders to estimate the probability of an N-gram with zero frequency.The result is better and lesser when compared to the bigram guesser.
